In [26]:
import os
import json

from config import MAIN_DIR
from pydantic import BaseModel
from llama_index.program import LLMTextCompletionProgram
from llama_index.output_parsers import PydanticOutputParser
from llama_index.llms.openai import OpenAI
from tqdm import tqdm

In [ ]:
with open(os.path.join(MAIN_DIR, "auth", "api_keys.json"), "r") as f:
    api_keys = json.load(f)
    
os.environ["OPENAI_API_KEY"] = api_keys["OPENAI_API_KEY"]

In [5]:
class PatientProfile(BaseModel):
    patient_facts: str
    scan_order: str

In [29]:
extract_template = """You are a radiologist expert. Do not make up additional information.
=========
TASK: You are given a PATIENT PROFILE containing the patient information, medical history, previous diagnosis and the scan order given by clinicians.
You need to extract the following information from the PATIENT PROFILE:
1. Facts related to the patients information, including demographics/previous diagnosis/symptoms
2. Scan order and clinicians suspicions for scan
=========
EXAMPLE:
PATIENT PROFILE: 78 year old Indian Female. Past medical history of diabetes mellitus and hypertension on follow up with polyclinic. Tripped and fell, subsequently unable to walk due to left hip pain. On examination: left limb foreshortening, externally rotated, reduced range of motion due to pain. Nil imaging performed thus far. MRI pelvis and left hip without IV contrast to assess for pelvic/hip fracture and alignment.
ANSWER:
- Patient facts: 78 year old Indian Female. Past medical history of diabetes mellitus and hypertension on follow up with polyclinic. Tripped and fell, subsequently unable to walk due to left hip pain. On examination: left limb foreshortening, externally rotated, reduced range of motion due to pain. Nil imaging performed thus far.
- Scan order: MRI pelvis and left hip without IV contrast to assess for pelvic/hip fracture and alignment.
=========
PATIENT PROFILE: {patient_profile}
"""

llm = OpenAI(model="gpt-4", temperature=0, max_tokens=256)

extract_program = LLMTextCompletionProgram.from_defaults(
    output_parser=PydanticOutputParser(output_cls=PatientProfile),
    prompt_template_str=extract_template, verbose=False,
    llm = llm
)

In [32]:
profiles = [
"48 year old Caucasian male.  MRI right foot without and with IV contrast for ?osteonecrosis of navicular.  ",
"8 year old Indian male.  MRI left knee without IV contrast for trauma.",
"19 year old Chinese male. MRI right knee without IV contrast for proximal tibia fracture.",
"31 year old Malay female.  MRI left forearm without and with IV contrast for lump.",
"55 year old Chinese female.  MRI thoracic and lumbar spine without and with IV contrast for ?pathological vertebral compression fracture.",
"34 year old Indian male.  MRI lumbosacral spine without and with IV contrast for ?ankylosing spondylitis.",
"24 year old Indian female.  MRI left shoulder without IV contrast for ?Hill Sachs lesion.  ",
"22 year old Malay male.  MRI right shoulder arthrography for ?dislocation.",
"28 year old Bangladeshi male.  MRI left wrist for ?tendon injury.",
"58 year old Chinese male.  MRI right foot for ?Charcot foot.",
"18 year old Eurasian male.  Full time national service, in guards battalion.  No prior medical history.  Now presents with severe right foot pain after fast march.  Cannot walk without assistance.  On examination, mildly swollen right foot.  Tender on deep palpation of 2nd and 3rd metatarsals.  No prior imaging.  For MRI of mid and forefoot without IV contrast to assess for stress fracture.  ",
"23 year old Malay male.  Professional soccer player.  Diagnosed with stress fracture of right tibia 3 months ago.  Now pain free for 2 weeks.  On examination, no tendereness at shin.  No deformity.  For MRI of right leg without and with IV contrast to assess suitability to return to play.",
"69 year old Chinese male.  Past medical history of diabetes, hypertension, hyperlipidemia, peripheral vascular disease post multiple balloon angioplasties, chronic renal disease.  Now presenting with left foot swelling and redness for 2 weeks, painful to walk.  On examination, left foot swelling, erythema, no crepitus.  No prior imaging.  For MRI of the left foot without and with IV contrast to assess for cellulitis and osteomyelitis. ",
"30 year old Caucasian male.  No prior medical history.  Now coming into emergency room for road traffic accident due to drink driving.  On examination, patient is intoxicated and in cervical collar.  Chest and pelvic compression positive.  Ecchymosis over abdomen in seat-belt injury.  No prior imaging.  For MRI cervical spine without IV contrast to assess for spine fracture and cord injury.  ",
"41 year old Chinese male.  No prior medical history.  Smoker.  Now coming into emergency room for road traffic accident. Patient was driving company car when rear-ended by a truck.  Car hit road divider.  On examintion, patient is alert, but power in bilateral lower limbs is 2 out of 5.  Slightly lax anal tone on digital rectal examination.  X-rays show T12 chance fracture.  MRI thoracic and lumbar spine without IV contrast to assess for neurologic injury.  ",
"48 year old Indian male.  CEO of shipping company.  Recreational cricket player.  Prior history of fatty liver, hyperlipidemia on statins.   Now presenting with low back pain for 3 days post cricket match.  No radiation to groin or lower limbs.  Able to walk.  Urination and bowel motion are ok.  No fever.  On examination, straight leg raise test is positive.  Power in bilateral lower limbs is full.  Digital rectal examination shows good anal tone.  No prior imaging.  For MRI lumbar spine with IV contrast to assess for prolapsed intervertebral disc or muscle strain. ",
"73 year old Malay female.  Past history of cholecystectomy, fatty liver, obesity, hypertension, hyperlipidemia, osteoarthritis of bilateral knees post bilateral total knee replacement.  Now presenting with low back pain for 6 days.  No radiation to groin or lower limbs.  No difficulty passing urine or bowel motion.  On examination, mild step deformity at L2 region.  Power full in bilateral lower limbs.  Good anal tone on digital rectal examination.  No prior imaging.  For MIR lumbar spine without IV contrast to further assess low back pain.",
"46 year old Caucasian female.  History of systemic lupus erythematosus on follow up with rheumatology.  Now presenting with 5 day history of worsening numbness and tingling in all 4 limbs, feels uncoordinated and more difficult to control continence.  On examination, reduced pinprick sensation in bilateral hands and feets, power in all 4 limbs is 3 to 4 out of 5.  Anal tone is equivocally lax.  No sensory level.  No prior imaging. For MRI whole spine without and with IV contrast to assess for myelopathy.",
"73 year old Indian male, with history of end stage diabetic nephropathy on haemodialysis. Complains of lower back pain for the past 2 months with occasional night pain. Associated with radicular pain of left lower limb. no fever. Physical exam reveals mild tenderness at L3-L4 region but otherwise unremarkable. No prior imaging. MRI lumbar spine without and with IV contrast for possible spondylodiscitis.",
"70 year old malay male. History of diabetes, hyperlipidemia and hypertension. Worsening neck pain for past 6 months, no radiculopathy or night pain. No recent trauma. Clinical exam reveals mild midline tenderness over the cervical spine with limited range of motion, but no focal neurological deficit. MRI cervical spine without IV contrast for initial examination. ",
"60 year old Chinese female. No past medical history.  Left wrist swelling and redness for 1 week. No trauma.  No fever. On examination, left wrist is warm swollen and tender.  No prior imaging. MRI of the left wrist without and with IV contrasts for possible septic arthritis."
]

In [33]:
extracted_datas = []
for profile in tqdm(profiles, total=len(profiles)):
    response_obj = extract_program(patient_profile=profile)
    extracted_datas.append(response_obj)

100%|██████████| 21/21 [01:00<00:00,  2.87s/it]
